In [1]:
import duckdb
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import RobustScaler, PowerTransformer
import lightgbm as lgb
import warnings
warnings.filterwarnings('ignore')

In [ ]:

# Configuración
PERIODO_CORTE = 201912  # Entrenar hasta este periodo
TARGET_LAG = 2  # Predecir periodo +2

def load_and_prepare_data():
    """
    Carga y prepara los datos para el modelo
    """
    # Conectar a la base de datos DuckDB
    con = duckdb.connect(database='../input/db/labo3.duckdb')
    
    # Cargar datos de la tabla ventas_features_final
    query = """SELECT 
        antiguedad_cliente,
        antiguedad_producto,
        cust_request_qty,
        cust_request_tn,
        customer_id,
        periodo,
        plan_precios_cuidados,
        product_id,
        registro_sintetico,
        -- stock_final,
        tn,
        anio,
        cliente_recurrente,
        coef_variacion_6m,
        crecimiento_consistente,
        delta_request_1m,
        delta_request_3m,
        delta_request_6m,
        delta_stock_1m,
        delta_stock_3m,
        delta_tn_12m,
        delta_tn_1m,
        delta_tn_3m,
        delta_tn_6m,
        es_invierno,
        es_verano,
        fill_rate,
        is_max_tn_12m,
        is_max_tn_3m,
        is_max_tn_6m,
        is_min_tn_12m,
        is_min_tn_3m,
        is_min_tn_6m,
        ma_request_3m,
        ma_request_6m,
        ma_stock_3m,
        ma_tn_12m,
        ma_tn_3m,
        ma_tn_6m,
        mes,
        pct_change_tn_1m,
        pct_change_tn_3m,
        pct_change_vs_mismo_mes_anio_anterior,
        producto_maduro,
        ratio_tn_vs_2m_ahead,
        ratio_vs_ma_6m,
        request_tn_lag_1m,
        request_tn_lag_3m,
        request_tn_lag_6m,
        stock_lag_1m,
        stock_lag_3m,
        stock_turnover_ratio,
        tendencia_6m,
        tn_lag_12m,
        tn_lag_1m,
        tn_lag_3m,
        tn_lag_6m,
        tn_mismo_mes_anio_anterior,
        trimestre,
        volatilidad_tn_6m,
        -- clientes_total_producto,
        coef_variacion_estacional,
        desviacion_vs_promedio_historico_mes,
        es_cliente_principal_producto,
        es_producto_principal_cliente,
        indice_concentracion_producto,
        indice_estacionalidad,
        -- meses_desde_ultima_compra,
        -- participacion_cliente_en_producto,
        -- participacion_producto_en_cliente,
        patron_ciclico_3m,
        patron_ciclico_6m,
        periodicidad_promedio_compras,
        productos_total_cliente_periodo,
        promedio_anual,
        promedio_historico_mes,
        ranking_cliente_en_producto,
        ranking_producto_en_cliente,
        ratio_vs_promedio_otros_productos_cliente,
        stddev_historico_mes,
        -- tn_total_cliente_periodo,
        -- tn_total_producto_periodo,
        cat1,
        cat2,
        -- cat3,
        brand
        -- sku_size
    FROM ventas_features_final"""
    df = con.execute(query).df()
    
    # Cerrar conexión
    con.close()
    print(f"Datos originales: {df.shape}")
    print(f"Periodos únicos: {sorted(df['periodo'].unique())}")
    
    return df

def analyze_target_distribution(df):
    """
    Analiza la distribución del target para entender el problema
    """
    print("=== ANÁLISIS DEL TARGET ===")
    target = df['target_tn'].dropna()
    
    print(f"Target stats:")
    print(f"  Count: {len(target):,}")
    print(f"  Min: {target.min():.4f}")
    print(f"  Max: {target.max():.4f}")
    print(f"  Mean: {target.mean():.4f}")
    print(f"  Median: {target.median():.4f}")
    print(f"  Std: {target.std():.4f}")
    
    # Percentiles
    percentiles = [1, 5, 10, 25, 50, 75, 90, 95, 99]
    print(f"\nPercentiles:")
    for p in percentiles:
        print(f"  {p}%: {np.percentile(target, p):.4f}")
    
    # Valores negativos y ceros
    negative_count = (target < 0).sum()
    zero_count = (target == 0).sum()
    positive_count = (target > 0).sum()
    
    print(f"\nDistribución de valores:")
    print(f"  Negativos: {negative_count:,} ({negative_count/len(target)*100:.1f}%)")
    print(f"  Ceros: {zero_count:,} ({zero_count/len(target)*100:.1f}%)")
    print(f"  Positivos: {positive_count:,} ({positive_count/len(target)*100:.1f}%)")
    
    return target
def identify_problematic_features(train_data, target_col='target_tn'):
    """
    Identifica variables problemáticas que pueden causar overfitting
    """
    print("=== IDENTIFICACIÓN DE VARIABLES PROBLEMÁTICAS ===")
    
    # 1. Variables que pueden ser data leakage
    potential_leaks = []
    suspicious_keywords = ['total', 'sum', 'global', 'all', 'periodo_total']
    
    for col in train_data.columns:
        for keyword in suspicious_keywords:
            if keyword in col.lower():
                potential_leaks.append(col)
                break
    
    print(f"Posibles data leaks ({len(potential_leaks)}): {potential_leaks}")
    
    # 2. Variables con valores extremos
    numeric_cols = train_data.select_dtypes(include=[np.number]).columns
    extreme_vars = []
    
    for col in numeric_cols:
        if col != target_col and col in train_data.columns:
            q99 = train_data[col].quantile(0.99)
            q01 = train_data[col].quantile(0.01)
            range_ratio = q99 / (q01 + 1e-8)  # Evitar división por 0
            
            if range_ratio > 1000:  # Rango muy amplio
                extreme_vars.append((col, range_ratio))
    
    extreme_vars.sort(key=lambda x: x[1], reverse=True)
    print(f"\nVariables con rangos extremos (top 10):")
    for col, ratio in extreme_vars[:10]:
        print(f"  {col}: ratio={ratio:.1f}")
    
    # 3. Variables altamente correlacionadas con el target
    target_corr = []
    for col in numeric_cols:
        if col != target_col and col in train_data.columns:
            corr = train_data[col].corr(train_data[target_col])
            if not np.isnan(corr) and abs(corr) > 0.8:
                target_corr.append((col, corr))
    
    target_corr.sort(key=lambda x: abs(x[1]), reverse=True)
    print(f"\nVariables muy correlacionadas con target (>0.8):")
    for col, corr in target_corr:
        print(f"  {col}: corr={corr:.3f}")
    
    return {
        'potential_leaks': potential_leaks,
        'extreme_vars': [x[0] for x in extreme_vars],
        'high_corr': [x[0] for x in target_corr]
    }

def create_feature_exclusion_list(problematic_features):
    """
    Crea lista de variables a excluir
    """
    print("=== CREANDO LISTA DE EXCLUSIÓN ===")
    
    # Variables definitivamente problemáticas
    definite_excludes = [
        'tn_total_cliente_periodo',
        'tn_total_producto_periodo', 
        'clientes_total_producto',
        'productos_total_cliente_periodo'
    ]

    # Variables importantes que NUNCA debemos excluir
    keep_always = [
        'antiguedad_cliente',
        'antiguedad_producto',
        #'stock_final',
        'fill_rate',
        'cust_request_qty',
        'cust_request_tn'
    ]
    
    auto_excludes = []

    # Filtrar potential_leaks excluyendo las importantes
    filtered_leaks = [var for var in problematic_features['potential_leaks'] 
                      if var not in keep_always]
    auto_excludes.extend(filtered_leaks)
    
    # Solo las más extremas de las variables con rangos amplios (pero filtradas)
    filtered_extreme = [var for var in problematic_features['extreme_vars'][:5] 
                        if var not in keep_always]
    auto_excludes.extend(filtered_extreme)
    
    # Variables muy correlacionadas (posible leakage) - pero filtradas
    filtered_corr = [var for var in problematic_features['high_corr'] 
                     if var not in keep_always]
    auto_excludes.extend(filtered_corr)
    
    # Remover duplicados
    all_excludes = list(set(definite_excludes + auto_excludes))
    
    print(f"Variables a excluir ({len(all_excludes)}):")
    for var in sorted(all_excludes):
        print(f"  - {var}")
    
    return all_excludes

def robust_preprocessing(X_train, X_test, numeric_vars, extreme_threshold=1000):
    """
    Preprocesamiento robusto usando RobustScaler y transformaciones
    """
    print("=== PREPROCESAMIENTO ROBUSTO ===")
    
    X_train_processed = X_train.copy()
    X_test_processed = X_test.copy()
    
    # Identificar columnas numéricas válidas
    numeric_cols_present = [col for col in numeric_vars if col in X_train.columns]
    
    transformers = {}
    
    for col in numeric_cols_present:
        if X_train_processed[col].dtype in ['int64', 'float64', 'int32', 'float32']:
            
            # 1. Manejar valores extremos
            q99 = X_train_processed[col].quantile(0.99)
            q01 = X_train_processed[col].quantile(0.01)
            
            # Winsorización (clip outliers)
            X_train_processed[col] = X_train_processed[col].clip(q01, q99)
            X_test_processed[col] = X_test_processed[col].clip(q01, q99)
            
            # 2. Verificar si necesita transformación adicional
            col_range = q99 - q01
            col_std = X_train_processed[col].std()
            
            if col_range > extreme_threshold or col_std > extreme_threshold:
                print(f"  Aplicando transformación robusta a: {col}")
                
                # Usar RobustScaler (menos sensible a outliers)
                scaler = RobustScaler()
                X_train_processed[col] = scaler.fit_transform(X_train_processed[col].values.reshape(-1, 1)).flatten()
                X_test_processed[col] = scaler.transform(X_test_processed[col].values.reshape(-1, 1)).flatten()
                
                transformers[col] = scaler
            else:
                # StandardScaler normal para variables bien comportadas
                from sklearn.preprocessing import StandardScaler
                scaler = StandardScaler()
                X_train_processed[col] = scaler.fit_transform(X_train_processed[col].values.reshape(-1, 1)).flatten()
                X_test_processed[col] = scaler.transform(X_test_processed[col].values.reshape(-1, 1)).flatten()
                
                transformers[col] = scaler
    
    print(f"Procesadas {len(transformers)} variables numéricas")
    
    return X_train_processed, X_test_processed, transformers

def debug_feature_selection(train_data, feature_cols):
    """
    Debuggea qué variables se están incluyendo incorrectamente
    """
    print("=== DEBUG DE SELECCIÓN DE FEATURES ===")
    
    # Variables que DEFINITIVAMENTE no deberían estar
    forbidden_vars = ['product_id', 'customer_id', 'periodo', 'target_tn', 'tn']
    forbidden_found = []
    
    for var in forbidden_vars:
        if var in feature_cols:
            forbidden_found.append(var)
    
    if forbidden_found:
        print(f"🚨 ERROR: Variables prohibidas encontradas en features:")
        for var in forbidden_found:
            print(f"  - {var}")
    
    # Variables con nombres sospechosos
    suspicious_vars = []
    for var in feature_cols:
        if any(keyword in var.lower() for keyword in ['total', 'id', 'key_']):
            suspicious_vars.append(var)
    
    if suspicious_vars:
        print(f"⚠️  Variables sospechosas en features:")
        for var in suspicious_vars:
            print(f"  - {var}")
    
    print(f"\nTotal features seleccionadas: {len(feature_cols)}")
    print(f"Primeras 10 features: {feature_cols[:10]}")
    
    return forbidden_found, suspicious_vars
def prepare_features_fixed(df):
    """
    Función de preparación de features con exclusión FORZADA
    """
    print("Preparando features (versión corregida)...")
    
    # 1. EXCLUSIÓN FORZADA DE IDENTIFICADORES
    forced_excludes = [
        'product_id', 'customer_id', 'periodo',  # IDs
        'target_tn', 'tn',  # Target y variable original
        'key_customer_producto_periodo',  # Keys
        'key_periodo_customer_producto',
        'key_periodo_producto',
        'PVC', 'PVP', 'UVC', 'UVP', 'periodo_fecha'  # Fechas
    ]
    
    # 2. EXCLUSIÓN DE VARIABLES PROBLEMÁTICAS
    problematic_excludes = [
        'tn_total_cliente_periodo',
        'tn_total_producto_periodo', 
        'clientes_total_producto',
        'productos_total_cliente_periodo'
    ]
    
    # 3. Variables categóricas
    categorical_vars = ['cat1', 'cat2', 'cat3', 'brand']
    
    # 4. Combinar todas las exclusiones
    all_excludes = forced_excludes + problematic_excludes
    
    # 5. Variables numéricas = todo lo demás
    all_vars = set(df.columns)
    excluded_set = set(all_excludes)
    categorical_set = set(categorical_vars)
    
    numeric_vars = list(all_vars - excluded_set - categorical_set)
    
    # 6. Verificar que no hay contaminación
    contaminated = []
    for var in numeric_vars:
        if any(forbidden in var.lower() for forbidden in ['product_id', 'customer_id', 'periodo']):
            contaminated.append(var)
    
    if contaminated:
        print(f"🚨 ELIMINANDO variables contaminadas: {contaminated}")
        numeric_vars = [var for var in numeric_vars if var not in contaminated]
    
    print(f"Variables categóricas ({len(categorical_vars)}): {categorical_vars}")
    print(f"Variables numéricas ({len(numeric_vars)})")
    print(f"Variables excluidas ({len(all_excludes)})")
    print(f"VERIFICACIÓN: product_id excluido = {'product_id' not in numeric_vars}")
    print(f"VERIFICACIÓN: customer_id excluido = {'customer_id' not in numeric_vars}")
    print(f"VERIFICACIÓN: periodo excluido = {'periodo' not in numeric_vars}")
    
    return numeric_vars, categorical_vars, all_excludes



def prepare_features_improved(df):
    """
    Versión mejorada de prepare_features que excluye variables problemáticas
    """
    print("Preparando features (versión mejorada)...")
    
    # 1. Identificar variables problemáticas
    problematic = identify_problematic_features(df)
    
    # 2. Crear lista de exclusión
    auto_excludes = create_feature_exclusion_list(problematic)
    
    # 3. Variables a excluir (originales + automáticas)
    exclude_vars = [col for col in df.columns if col.startswith('key_')]
    exclude_vars.extend(['target_tn', 'tn'])
    
    # Excluir variables de fecha
    date_vars = ['PVC', 'PVP', 'UVC', 'UVP', 'periodo_fecha']
    exclude_vars.extend([col for col in date_vars if col in df.columns])

    id_vars = ['product_id', 'customer_id', 'periodo']
    exclude_vars.extend([col for col in id_vars if col in df.columns])
    
    # Agregar exclusiones automáticas
    exclude_vars.extend(auto_excludes)
    
    # Remover duplicados
    exclude_vars = list(set(exclude_vars))
    
    # Variables categóricas
    categorical_vars = ['cat1', 'cat2', 'cat3', 'brand']
    
    # Variables numéricas
    all_vars = set(df.columns)
    numeric_vars = list(all_vars - set(exclude_vars) - set(categorical_vars))
    
    print(f"Variables categóricas ({len(categorical_vars)}): {categorical_vars}")
    print(f"Variables numéricas ({len(numeric_vars)}): {numeric_vars[:5]}...")
    print(f"Variables excluidas ({len(exclude_vars)}): {auto_excludes[:5]}...")
    
    return numeric_vars, categorical_vars, exclude_vars

def apply_same_preprocessing(X_new, transformers, numeric_vars):
    """
    Aplica las mismas transformaciones del entrenamiento a nuevos datos
    """
    X_processed = X_new.copy()
    
    for col in numeric_vars:
        if col in transformers and col in X_processed.columns:
            if X_processed[col].dtype in ['int64', 'float64', 'int32', 'float32']:
                # Aplicar la misma transformación que se usó en entrenamiento
                scaler = transformers[col]
                X_processed[col] = scaler.transform(X_processed[col].values.reshape(-1, 1)).flatten()
    
    return X_processed

def apply_post_prediction_constraints(predictions):
    """
    Aplica restricciones post-predicción para asegurar valores válidos
    """
    print("Aplicando restricciones post-predicción...")
    
    original_min = predictions.min()
    original_max = predictions.max()
    negative_count = (predictions < 0).sum()
    
    print(f"Predicciones originales - Min: {original_min:.4f}, Max: {original_max:.4f}")
    print(f"Predicciones negativas: {negative_count} ({negative_count/len(predictions)*100:.1f}%)")
    
    # Clip a 0 (evita valores negativos)
    predictions_clipped = np.maximum(predictions, 0)
    
    print(f"Después del clip - Min: {predictions_clipped.min():.4f}, Max: {predictions_clipped.max():.4f}")
    
    return predictions_clipped



def create_target_variable(df):
    """
    Crea la variable objetivo (tn en periodo +2) agrupada por product_id
    """
    print("Creando variable objetivo...")
    
    # Agregar tn por product_id y periodo
    df_agg = df.groupby(['product_id', 'periodo']).agg({
        'tn': 'sum'
    }).reset_index()
    
    # Ordenar por product_id y periodo
    df_agg = df_agg.sort_values(['product_id', 'periodo'])
    
    # Crear target variable (tn en periodo +2)
    df_agg['target_tn'] = df_agg.groupby('product_id')['tn'].shift(-TARGET_LAG)
    
    # Merge de vuelta con el dataset original
    df = df.merge(df_agg[['product_id', 'periodo', 'target_tn']], 
                  on=['product_id', 'periodo'], how='left')
    
    print(f"Registros con target válido: {df['target_tn'].notna().sum()}")
    
    return df

def prepare_features(df):
    """
    Prepara las features excluyendo variables key_*, fechas y separando categóricas
    """
    print("Preparando features...")
    
    # Variables a excluir
    exclude_vars = [col for col in df.columns if col.startswith('key_')]
    exclude_vars.extend(['target_tn', 'tn'])  # Excluir también el target y la variable original
    
    # Excluir variables de fecha que no son útiles como features numéricas
    date_vars = ['PVC', 'PVP', 'UVC', 'UVP', 'periodo_fecha']
    exclude_vars.extend([col for col in date_vars if col in df.columns])
    
    # Variables categóricas
    categorical_vars = ['cat1', 'cat2', 'cat3', 'brand']
    
    # Variables numéricas (todas las demás excepto las excluidas)
    all_vars = set(df.columns)
    numeric_vars = list(all_vars - set(exclude_vars) - set(categorical_vars))
    
    print(f"Variables categóricas ({len(categorical_vars)}): {categorical_vars}")
    print(f"Variables numéricas ({len(numeric_vars)}): {numeric_vars[:10]}...")  # Mostrar solo las primeras 10
    print(f"Variables excluidas ({len(exclude_vars)}): {exclude_vars}")
    
    return numeric_vars, categorical_vars, exclude_vars

def encode_categorical_variables(df, categorical_vars):
    """
    Codifica variables categóricas usando LabelEncoder
    """
    print("Codificando variables categóricas...")
    
    label_encoders = {}
    df_encoded = df.copy()
    
    for col in categorical_vars:
        if col in df.columns:
            le = LabelEncoder()
            # Manejar valores nulos
            df_encoded[col] = df_encoded[col].fillna('UNKNOWN')
            df_encoded[col] = le.fit_transform(df_encoded[col].astype(str))
            label_encoders[col] = le
            print(f"  {col}: {len(le.classes_)} categorías únicas")
    
    return df_encoded, label_encoders

def clean_numeric_data(df):
    """
    Limpia y convierte datos numéricos para evitar problemas
    """
    print("Limpiando datos numéricos...")
    
    df_clean = df.copy()
    
    # Convertir plan_precios_cuidados de bool a int
    if 'plan_precios_cuidados' in df_clean.columns:
        df_clean['plan_precios_cuidados'] = df_clean['plan_precios_cuidados'].astype(int)
        print("  Convertido plan_precios_cuidados de bool a int")
    
    # Identificar columnas numéricas
    numeric_columns = df_clean.select_dtypes(include=[np.number]).columns
    
    for col in numeric_columns:
        # Reemplazar infinitos por NaN
        df_clean[col] = df_clean[col].replace([np.inf, -np.inf], np.nan)
        
        # Convertir float64 a float32 para ahorrar memoria y evitar problemas de precisión
        if df_clean[col].dtype == 'float64':
            df_clean[col] = df_clean[col].astype('float32')
    
    print(f"  Limpiadas {len(numeric_columns)} columnas numéricas")
    print("  Convertidos float64 -> float32")
    print("  Reemplazados valores infinitos por NaN")
    
    return df_clean

def standardize_numeric_features(X_train, X_test, numeric_vars):
    """
    Estandariza variables numéricas
    """
    print("Estandarizando variables numéricas...")
    
    scaler = StandardScaler()
    
    # Copiar los datasets
    X_train_scaled = X_train.copy()
    X_test_scaled = X_test.copy()
    
    # Manejar valores nulos antes de estandarizar
    X_train_scaled = X_train_scaled.fillna(0)
    X_test_scaled = X_test_scaled.fillna(0)
    
    # Identificar columnas numéricas que existen en los datos
    numeric_cols_present = [col for col in numeric_vars if col in X_train.columns]
    
    # Verificar que las columnas sean realmente numéricas y no tengan problemas
    numeric_cols_valid = []
    for col in numeric_cols_present:
        if X_train_scaled[col].dtype in ['int64', 'float64', 'int32', 'float32']:
            # Verificar que no haya infinitos o valores muy grandes
            if not (np.isinf(X_train_scaled[col]).any() or np.isinf(X_test_scaled[col]).any()):
                # Verificar que no haya valores extremadamente grandes
                max_val = max(X_train_scaled[col].abs().max(), X_test_scaled[col].abs().max())
                if max_val < 1e10:  # Límite razonable
                    numeric_cols_valid.append(col)
                else:
                    print(f"  Saltando {col}: valores muy grandes (max: {max_val:.2e})")
            else:
                print(f"  Saltando {col}: contiene valores infinitos")
        else:
            print(f"  Saltando columna no numérica: {col} (tipo: {X_train_scaled[col].dtype})")
    
    if numeric_cols_valid:
        # Estandarizar solo las columnas numéricas válidas
        X_train_scaled[numeric_cols_valid] = scaler.fit_transform(X_train_scaled[numeric_cols_valid])
        X_test_scaled[numeric_cols_valid] = scaler.transform(X_test_scaled[numeric_cols_valid])
        print(f"Estandarizadas {len(numeric_cols_valid)} variables numéricas")
    else:
        print("No se encontraron variables numéricas válidas para estandarizar")
    
    return X_train_scaled, X_test_scaled, scaler

def train_lgbm_model(X_train, y_train, X_val, y_val, categorical_features):
    """
    Entrena modelo LightGBM con parámetros mejorados
    """
    print("Entrenando modelo LightGBM mejorado...")
    
    # Parámetros mejorados
    params = {
        'objective': 'regression',
        'metric': 'rmse',
        'boosting_type': 'gbdt',
        'num_leaves': 50,  # Aumentado
        'learning_rate': 0.03,  # Reducido
        'feature_fraction': 0.8,
        'bagging_fraction': 0.8,
        'bagging_freq': 5,
        'min_data_in_leaf': 100,  # NUEVO - evita overfitting
        'lambda_l1': 0.1,  # NUEVO - regularización
        'lambda_l2': 0.1,  # NUEVO - regularización
        'verbose': -1,
        'random_state': 42,
        'force_col_wise': True
    }
    
    # Crear datasets de LightGBM
    train_data = lgb.Dataset(X_train, label=y_train, categorical_feature=categorical_features)
    val_data = lgb.Dataset(X_val, label=y_val, categorical_feature=categorical_features, reference=train_data)
    
    # Entrenar modelo
    model = lgb.train(
        params,
        train_data,
        valid_sets=[train_data, val_data],
        valid_names=['train', 'val'],
        num_boost_round=2000,  # Aumentado
        callbacks=[lgb.early_stopping(stopping_rounds=200), lgb.log_evaluation(period=100)]
    )
    
    return model

def evaluate_model(model, X_test, y_test):
    """
    Evalúa el modelo con mejoras
    """
    print("Evaluando modelo...")
    
    # Predicciones base
    y_pred = model.predict(X_test)
    
    # Aplicar restricciones (clip negativo a 0)
    y_pred_final = apply_post_prediction_constraints(y_pred)
    
    # Métricas
    mae = mean_absolute_error(y_test, y_pred_final)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred_final))
    r2 = r2_score(y_test, y_pred_final)
    
    print(f"\nMétricas del modelo:")
    print(f"MAE: {mae:.4f}")
    print(f"RMSE: {rmse:.4f}")
    print(f"R²: {r2:.4f}")
    
    return {'mae': mae, 'rmse': rmse, 'r2': r2, 'predictions': y_pred_final}

def get_feature_importance(model, feature_names):
    """
    Obtiene importancia de features
    """
    importance = model.feature_importance(importance_type='gain')
    feature_imp = pd.DataFrame({
        'feature': feature_names,
        'importance': importance
    }).sort_values('importance', ascending=False)
    
    print("\nTop 15 features más importantes:")
    print(feature_imp.head(15))
    
    return feature_imp

def main():
    """
    Función principal
    """
    print("=== MODELO LGBM PARA PREDICCIÓN DE TONELADAS ===\n")
    
    # 1. Cargar datos
    df = load_and_prepare_data()
    
    # 2. Crear variable objetivo
    df = create_target_variable(df)
    
    # 2.5. Limpiar datos numéricos (nueva función)
    df = clean_numeric_data(df)
    
    # 3. Filtrar datos para entrenamiento (hasta PERIODO_CORTE)
    train_data = df[df['periodo'] <= PERIODO_CORTE].copy()
    print(f"\nDatos de entrenamiento: {train_data.shape}")
    
    # 4. Remover registros sin target
    train_data = train_data.dropna(subset=['target_tn'])
    print(f"Datos después de remover NaN en target: {train_data.shape}")
    
    # 4.5. Analizar distribución del target ← NUEVO
    target_analysis = analyze_target_distribution(train_data)

    # 5. Preparar features
    numeric_vars, categorical_vars, exclude_vars = prepare_features_fixed(train_data)
    #numeric_vars, categorical_vars, exclude_vars = prepare_features(train_data)
    #numeric_vars, categorical_vars, exclude_vars = prepare_features_improved(train_data)
    
    # 6. Codificar variables categóricas
    train_data_encoded, label_encoders = encode_categorical_variables(train_data, categorical_vars)
    
    # 7. Seleccionar features finales
    feature_cols = numeric_vars + categorical_vars
    feature_cols = [col for col in feature_cols if col in train_data_encoded.columns]
    
    print(f"\nFeatures finales: {len(feature_cols)}")
    
    # 8. Preparar X y y
    forbidden_found, suspicious_vars = debug_feature_selection(train_data, feature_cols)

       
    if forbidden_found:
        print("🛑 DETENIENDO: Variables prohibidas detectadas")
        return None

    X = train_data_encoded[feature_cols].copy()
    y = train_data_encoded['target_tn'].copy()
    
    # Manejar valores nulos en X
    X = X.fillna(0)
    
    print(f"Shape final - X: {X.shape}, y: {y.shape}")
    
    # 9. Split train/test
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, stratify=None
    )
    
    # 10. Split train/validation
    X_train, X_val, y_train, y_val = train_test_split(
        X_train, y_train, test_size=0.2, random_state=42
    )
    
    print(f"Train: {X_train.shape}, Val: {X_val.shape}, Test: {X_test.shape}")
    
    # 11. Estandarizar variables numéricas
    X_train_scaled, X_val_scaled, scaler = standardize_numeric_features(X_train, X_val, numeric_vars)
    #X_train_scaled, X_val_scaled, transformers = robust_preprocessing(X_train, X_val, numeric_vars)
    X_test_scaled, _, _ = standardize_numeric_features(X_test, X_test, numeric_vars)
    #X_test_scaled = apply_same_preprocessing(X_test, transformers, numeric_vars)
    
    # 12. Identificar features categóricas para LightGBM
    categorical_indices = [i for i, col in enumerate(feature_cols) if col in categorical_vars]
    
    # 13. Entrenar modelo
    model = train_lgbm_model(
        X_train_scaled, y_train, 
        X_val_scaled, y_val, 
        categorical_indices
    )
    
    # 14. Evaluar modelo
    results = evaluate_model(model, X_test_scaled, y_test)
    
    # 15. Feature importance
    feature_importance = get_feature_importance(model, feature_cols)
    
    # 16. Guardar modelo y objetos necesarios
    print("\nGuardando modelo...")
    model.save_model('../output/lgbm/02_lgbm_model_v2.txt')
    
    # Guardar escalador y encoders
    import pickle
    with open('../output/lgbm/02_lgbm_model_v2.pkl', 'wb') as f:
        pickle.dump({
            #'transformers': transformers,
            'scaler': scaler,
            'label_encoders': label_encoders,
            'feature_cols': feature_cols,
            'categorical_vars': categorical_vars,
            'numeric_vars': numeric_vars
        }, f)
    
    print("Modelo guardado como '02_lgbm_model_v2.txt'")
    print("Objetos auxiliares guardados como '02_lgbm_model_v2.pkl'")
    
    return model, results, feature_importance

if __name__ == "__main__":
    model, results, feature_importance = main()

=== MODELO LGBM PARA PREDICCIÓN DE TONELADAS ===



FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Datos originales: (17021654, 38)
Periodos únicos: [201701, 201702, 201703, 201704, 201705, 201706, 201707, 201708, 201709, 201710, 201711, 201712, 201801, 201802, 201803, 201804, 201805, 201806, 201807, 201808, 201809, 201810, 201811, 201812, 201901, 201902, 201903, 201904, 201905, 201906, 201907, 201908, 201909, 201910, 201911, 201912]
Creando variable objetivo...
Registros con target válido: 15628837
Limpiando datos numéricos...
  Convertido plan_precios_cuidados de bool a int
  Limpiadas 36 columnas numéricas
  Convertidos float64 -> float32
  Reemplazados valores infinitos por NaN

Datos de entrenamiento: (17021654, 39)
Datos después de remover NaN en target: (15628837, 39)
=== ANÁLISIS DEL TARGET ===
Target stats:
  Count: 15,628,837
  Min: 0.0000
  Max: 2295.1982
  Mean: 42.6592
  Median: 9.6803
  Std: 107.7060

Percentiles:
  1%: 0.0064
  5%: 0.1873
  10%: 0.6104
  25%: 2.2491
  50%: 9.6803
  75%: 30.0179
  90%: 106.0346
  95%: 191.1998
  99%: 547.6251

Distribución de valores:


In [9]:
# =============================================================================
# CHUNK DE PREDICCIÓN PARA PRODUCTOS ESPECÍFICOS
# =============================================================================

import duckdb
import pandas as pd
import numpy as np
import lightgbm as lgb
import pickle
from sklearn.preprocessing import StandardScaler, LabelEncoder
import warnings
warnings.filterwarnings('ignore')

def load_productos_a_predecir():
    """
    Carga la lista de productos a predecir desde DuckDB
    """
    print("Cargando productos a predecir...")
    
    # Conectar a la base de datos DuckDB
    con = duckdb.connect(database='../input/db/labo3.duckdb')
    
    # Cargar productos a predecir
    query = "SELECT * FROM tb_productos_a_predecir"
    productos_df = con.execute(query).df()
    
    con.close()
    
    print(f"Productos a predecir: {len(productos_df)}")
    print(f"Product_ids únicos: {productos_df['product_id'].nunique()}")
    
    return productos_df

def load_latest_data_for_prediction():
    """
    Carga los datos más recientes para hacer predicciones
    """
    print("Cargando datos más recientes para predicción...")
    
    # Conectar a la base de datos DuckDB
    con = duckdb.connect(database='../input/db/labo3.duckdb')
    
    # Cargar datos del periodo más reciente (201912 para predecir 202002)
    # Necesitamos datos del periodo 201912 para predecir periodo +2 (202002)
    query = """
    SELECT * 
    FROM ventas_features_final 
    WHERE periodo = 201912
    """
    df = con.execute(query).df()
    
    con.close()
    
    print(f"Datos del periodo 201912: {df.shape}")
    
    return df

def load_model_and_objects():
    """
    Carga el modelo entrenado y objetos auxiliares
    """
    print("Cargando modelo y objetos auxiliares...")
    
    # Cargar modelo LightGBM
    model = lgb.Booster(model_file='../output/lgbm/02_lgbm_model_v2.txt')
    
    # Cargar objetos auxiliares
    with open('../output/lgbm/02_lgbm_model_v2.pkl', 'rb') as f:
        objects = pickle.load(f)
    
    return model, objects

def clean_numeric_data_prediction(df):
    """
    Aplica la misma limpieza de datos numéricos que en entrenamiento
    """
    print("Limpiando datos numéricos para predicción...")
    
    df_clean = df.copy()
    
    # Convertir plan_precios_cuidados de bool a int
    if 'plan_precios_cuidados' in df_clean.columns:
        df_clean['plan_precios_cuidados'] = df_clean['plan_precios_cuidados'].astype(int)
    
    # Identificar columnas numéricas
    numeric_columns = df_clean.select_dtypes(include=[np.number]).columns
    
    for col in numeric_columns:
        # Reemplazar infinitos por NaN
        df_clean[col] = df_clean[col].replace([np.inf, -np.inf], np.nan)
        
        # Convertir float64 a float32
        if df_clean[col].dtype == 'float64':
            df_clean[col] = df_clean[col].astype('float32')
    
    print(f"  Limpiadas {len(numeric_columns)} columnas numéricas")
    
    return df_clean

def prepare_prediction_data(df, objects):
    """
    Prepara datos para predicción usando los mismos pasos del entrenamiento
    PREDICE PARA TODOS LOS PRODUCTOS (no filtra)
    """
    print("Preparando datos para predicción...")
    
    # NO filtrar - usar todos los productos para predicción
    df_filtered = df.copy()
    
    print(f"Datos para predicción (todos los productos): {df_filtered.shape}")
    
    if df_filtered.empty:
        print("¡ADVERTENCIA! No se encontraron datos para predicción.")
        return None
    
    # Aplicar limpieza de datos
    df_filtered = clean_numeric_data_prediction(df_filtered)
    
    scaler = objects['scaler']
    label_encoders = objects['label_encoders']
    feature_cols = objects['feature_cols']
    categorical_vars = objects['categorical_vars']
    numeric_vars = objects['numeric_vars']
    
    # Codificar variables categóricas
    for col in categorical_vars:
        if col in df_filtered.columns and col in label_encoders:
            le = label_encoders[col]
            df_filtered[col] = df_filtered[col].fillna('UNKNOWN')
            
            # Manejar categorías no vistas durante el entrenamiento
            unknown_mask = ~df_filtered[col].astype(str).isin(le.classes_)
            df_filtered[col] = df_filtered[col].astype(str)
            df_filtered.loc[unknown_mask, col] = 'UNKNOWN'
            
            # Si 'UNKNOWN' no está en las clases, usar la primera clase
            if 'UNKNOWN' not in le.classes_:
                df_filtered.loc[unknown_mask, col] = le.classes_[0]
            
            df_filtered[col] = le.transform(df_filtered[col])
    
    # Seleccionar features
    missing_features = [col for col in feature_cols if col not in df_filtered.columns]
    if missing_features:
        print(f"¡ADVERTENCIA! Features faltantes: {missing_features}")
        # Crear columnas faltantes con valor 0
        for col in missing_features:
            df_filtered[col] = 0
    
    X_pred = df_filtered[feature_cols].copy()
    
    # Manejar valores nulos
    X_pred = X_pred.fillna(0)
    
    # Estandarizar variables numéricas usando los mismos pasos del entrenamiento
    numeric_cols_present = [col for col in numeric_vars if col in X_pred.columns]
    numeric_cols_valid = []
    
    for col in numeric_cols_present:
        if X_pred[col].dtype in ['int64', 'float64', 'int32', 'float32']:
            if not np.isinf(X_pred[col]).any():
                max_val = X_pred[col].abs().max()
                if max_val < 1e10:
                    numeric_cols_valid.append(col)
    
    if numeric_cols_valid:
        X_pred[numeric_cols_valid] = scaler.transform(X_pred[numeric_cols_valid])
    
    # Conservar información para el resultado final
    result_info = df_filtered[['customer_id', 'product_id']].copy()
    
    return X_pred, result_info

def make_predictions_for_products():
    """
    Función principal para hacer predicciones para productos específicos
    """
    print("=== PREDICCIÓN PARA PRODUCTOS ESPECÍFICOS ===\n")
    
    # 1. Cargar productos a predecir
    productos_a_predecir = load_productos_a_predecir()
    
    # 2. Cargar modelo y objetos
    model, objects = load_model_and_objects()
    
    # 3. Cargar datos más recientes
    df_latest = load_latest_data_for_prediction()
    
    # 4. Preparar datos para predicción (TODOS los productos)
    prediction_result = prepare_prediction_data(df_latest, objects)
    
    if prediction_result is None:
        print("No se pudieron preparar los datos para predicción.")
        return None
    
    X_pred, result_info = prediction_result
    
    # 5. Hacer predicciones PARA TODOS LOS PRODUCTOS
    print("Generando predicciones mejoradas para todos los productos...")
    predictions_raw = model.predict(X_pred)
    predictions = apply_post_prediction_constraints(predictions_raw)  # ← NUEVO
    
    # 6. Agregar predicciones al resultado
    result_info['predicted_tn'] = predictions
    
    # 7. Agrupar por product_id y sumar las predicciones (suma por todos los customer_id)
    print("Agrupando predicciones por product_id...")
    all_predictions = result_info.groupby('product_id').agg({
        'predicted_tn': 'sum'
    }).reset_index()
    
    # Renombrar columna para claridad
    all_predictions = all_predictions.rename(columns={'predicted_tn': 'tn'})
    
    print(f"Predicciones generadas para {len(all_predictions)} productos en total")
    
    # 8. FILTRAR RESULTADOS: Solo enviar productos que están en tb_productos_a_predecir
    product_ids_to_send = productos_a_predecir['product_id'].unique()
    final_predictions = all_predictions[
        all_predictions['product_id'].isin(product_ids_to_send)
    ].copy()
    
    # Ordenar por tn descendente
    final_predictions = final_predictions.sort_values('tn', ascending=False)
    
    print(f"\nResultados finales (filtrados):")
    print(f"Productos en tb_productos_a_predecir: {len(product_ids_to_send)}")
    print(f"Productos con predicción disponible: {len(final_predictions)}")
    print(f"Total tn predichas (solo productos solicitados): {final_predictions['tn'].sum():.2f}")
    print(f"Promedio tn por producto: {final_predictions['tn'].mean():.2f}")
    
    # Mostrar top 10
    print("\nTop 10 productos solicitados con mayor predicción:")
    print(final_predictions.head(10))
    
    # 9. Verificar que tenemos todos los productos solicitados
    productos_solicitados = set(product_ids_to_send)
    productos_predichos = set(final_predictions['product_id'].unique())
    productos_faltantes = productos_solicitados - productos_predichos
    
    if productos_faltantes:
        print(f"\n¡ADVERTENCIA! Productos solicitados sin datos en 201912: {len(productos_faltantes)}")
        print(f"Product_ids faltantes: {list(productos_faltantes)[:10]}...")  # Mostrar solo los primeros 10
        
        # Agregar productos faltantes con tn = 0
        for product_id in productos_faltantes:
            final_predictions = pd.concat([
                final_predictions,
                pd.DataFrame({'product_id': [product_id], 'tn': [0.0]})
            ], ignore_index=True)
    
    print(f"\nPredicciones finales enviadas: {len(final_predictions)} productos")
    
    # 10. Guardar resultados
    output_file = '../output/lgbm/02_lgbm_model_v2.csv'
    final_predictions.to_csv(output_file, index=False)
    print(f"Predicciones guardadas en: {output_file}")
    
    # Opcional: Guardar también TODAS las predicciones para análisis
    #all_output_file = 'predicciones_todos_los_productos.csv'
    #all_predictions.sort_values('tn', ascending=False).to_csv(all_output_file, index=False)
    #print(f"TODAS las predicciones guardadas en: {all_output_file}")
    
    return final_predictions

# Ejecutar predicciones
if __name__ == "__main__":
    predicciones = make_predictions_for_products()

=== PREDICCIÓN PARA PRODUCTOS ESPECÍFICOS ===

Cargando productos a predecir...
Productos a predecir: 780
Product_ids únicos: 780
Cargando modelo y objetos auxiliares...
Cargando datos más recientes para predicción...
Datos del periodo 201912: (553419, 98)
Preparando datos para predicción...
Datos para predicción (todos los productos): (553419, 98)
Limpiando datos numéricos para predicción...
  Limpiadas 85 columnas numéricas
Generando predicciones mejoradas para todos los productos...
Aplicando restricciones post-predicción...
Predicciones originales - Min: -363.2131, Max: 1435.9539
Predicciones negativas: 59310 (10.7%)
Después del clip - Min: 0.0000, Max: 1435.9539
Agrupando predicciones por product_id...
Predicciones generadas para 927 productos en total

Resultados finales (filtrados):
Productos en tb_productos_a_predecir: 780
Productos con predicción disponible: 780
Total tn predichas (solo productos solicitados): 17512233.59
Promedio tn por producto: 22451.58

Top 10 productos so